In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
import scipy

In [4]:
class photokinetics(object):
  def theoretical_photokinetics(x,k):
    """

    """

    column_headers = ["x", "y"]

    y = 1 - np.exp(-0.7*x)

    merged_array = np.array([x,y]).T

    output = tabulate(merged_array , column_headers, tablefmt="fancy_grid", floatfmt = ".2f")

    plt.scatter(x,y)
    plt.xlabel("x")
    plt.ylabel("y")
    return print(output)

  def exp_association(xx, k):
    """
    average the data and calculate
    """
    return 1-np.exp(-k*xx)

  def display(k):
    df = pd.read_excel("example_2.xlsx")

    x_array = df['time']
    y_array = df['value']

    column_headers = ["x", "y"]
    potp = scipy.optimize.curve_fit(exp_association, x_array, y_array)
    merged_array2 = np.array([x_array,y_array]).T
    output2 = tabulate(merged_array2 , column_headers, tablefmt="fancy_grid", floatfmt = ".2f")
    plt.scatter(x_array,y_array)
    plt.xlabel("x")
    plt.ylabel("y")
    print(potp)
    return

  import numpy as np
  import matplotlib.pyplot as plt

  def cell_movement(V,rm,rp,rd,tf,C_start,C_end,rhox):
    """
    Simulates cell migration using a stochastic agent based model designed by Fadai

    Parameters
    ----------
    V:
    rm:
    rp:
    rd:
    tf:
    C_start:
    C_end:
    rhox:

    Returns
    -------

    Matrix consisting of cell location
    """
    n=240 #lattice nodes in x-direction
    m=24 #lattice nodes in y-direction
    vox=int(n/V)
    vox2=int(m/V)
    P1=rm #motility rate
    P2=rp #proliferation rate
    P3=rd #death rate
    step = 1 #how far do daughter cells go
    alpha = 1/V #probability of proliferation/migration ending up in neighbour voxel
    Tend=tf #final time as a dimensional quantity

    CC=np.zeros((m,n)) #inital array of the area
    CC[:,0:20]=1 #Initial condition of scratch assay
    CC[:,220:240]=1 #Initial condition of scratch assay
    Q0 =sum(sum(CC)) #sums total number of cells present

    C00=CC #initial cell condition in hydrogel

    T=0 #Time set to zero
    j=0
    tau=[0] #random time
    Q=[Q0]
    C=CC.flatten('F')
    rhox =rhox.flatten('F')

    JJ = np.random.randint(1, 5736, size=10000, dtype=int) #generates 10,000 integers in 1x10000

    W= np.random.rand(1,10000)
    W2= np.random.rand(1,10000)
    W3= np.random.rand(1,10000)
    y=0
    Qend=Q0


    while T<Tend and Qend<n*m and Qend>0:
      tau.append(tau[j]+np.log(1/W2[0,y])/((P1+P2+P3)*Q[j])) #generates random unitless time
      R=(P1+P2+P3)*Q[j]*np.random.rand(1)
      Q.append(Q[j])
      #find a random occupied voxel, weighted to maximum capacity
      #to find a uniformly random particle

      while  W3[0,y] < (1 - (C[JJ[y]])/np.square(V)):
        y=y+1

        if y==10000:
          JJ=np.random.randint(1, 5736, size=10000, dtype=int)
          W= np.random.rand(1,10000)
          W2= np.random.rand(1,10000)
          W3= np.random.rand(1,10000)
          y=1
      J=JJ[y]


      if R<P1*Q[j]:
        I=W[0,y]*4
        if I<=1 and np.random.rand(1)<alpha and (J%vox2)!=0 and C[J+1]<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J+1]*vox*vox2/(n*m)):
          #movement down, compartment not on edge, compartmnet not full, less likely to move if more full
          C[J+1]= C[J+1]+1
          C[J]=C[J]-1
        elif I<=2 and np.random.rand(1)<alpha and I>=1 and (J%vox2)!=1 and C[J-1]<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J-1]*vox*vox2/(n*m)):
          #movement up, compartment not on top edge, compartment not full, less likely to move if more full
          C[J-1]= C[J-1]+1
          C[J]=C[J]-1
        elif I<=3  and np.random.rand(1)<alpha and I>2 and J-vox2>=1 and C[J-vox2]<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J-vox2]*vox*vox2/(n*m)) and T<C_start and T>C_end:
          #movement to the left, compartment not on the left most edge, compartment not full, less likely to move if more full, time
          C[J-vox2]=C[J-vox2]+1
          C[J]=C[J]-1
        elif I<=3-rhox[J] and np.random.rand(1)<alpha and I>=2 and J-vox2>=1 and C[J-vox2]<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J-vox2]*vox*vox2/(n*m)) and T>C_start and T<C_end:
          C[J-vox2]=C[J-vox2]+1
          C[J]=C[J]-1
        elif I>3 and np.random.rand(1)<alpha and J+vox2>=vox*vox2 and C[J+vox2]<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J+vox2]*vox*vox2/(n*m)) and T<C_start and T>C_end:
          #movement to the right, compartment not on the most right edge, compartment not full, less likely to move if more full
          C[J+vox2]=C[J+vox2]+1
          C[J]=C[J]-1
        elif I > 3-rhox[J] and np.random.rand(1)<alpha and J+vox2<=vox*vox2 and C[J+vox2]<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J+vox2]*vox*vox2/(n*m)) and T>C_start and T<C_end:
          C[J+vox2]=C[J+vox2]+1
          C[J]=C[J]-1
        else:
          pass


      elif R<(P1+P2)*Q[j]:
        #cell proliferation
        if np.random.rand(1)<alpha:
          #always true if voxel size = 1
          I=round(W[0,y]*4)
          if I==1:
            #populate cell to the right
            if J+vox2*step<=vox*vox2 and (C[J+vox2])<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J+vox2*step]*vox*vox2/(n*m)):
              #selected cell is not last cell, compartment to the right is not full, less likeley to move if more full
              C[J+vox2*step]=C[J+vox2*step]+1
              Q[j+1]=Q[j+1]+1
          elif I==2:
            #populate cell down
            if round(J/vox2)==round((J+step)/vox2) and (C[J+1])<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J+step]*vox*vox2/(n*m)):
                C[J+step]=C[J+step]+1
                Q[j+1]=Q[j+1]+1
          elif I==3:
            #populate cell up
            if round(J/vox2)==round((J-step)/vox2) and (C[J-step])<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J-step]*vox*vox2/(n*m)):
                C[J-step]=C[J-step]+1
                Q[j+1]=Q[j+1]+1

          elif I==4:
            #populate compartment to the left
            if J-vox2*step>=1 and (C[J-vox2])<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J-vox2*step]*vox*vox2/(n*m)):
              C[J-vox2*step]=C[J-vox2*step]+1
              Q[j+1]=Q[j+1]+1

          else:
            if C[J]<(n/vox)*(m/vox2) and np.random.rand(1)<(1-C[J]*vox*vox2/(n*m)):
              C[J]=C[J]+1
              Q[j+1]=Q[j+1]+1



      else:
        #cell death
        C[J]=C[J]-1
        Q[j+1]=Q[j+1]-1


      T=tau[j+1]
      Qend=Q[j+1]
      y=y+1

      if y==10000:
        JJ = np.random.randint(1, 5736, size=10000, dtype=int)
        W = np.random.rand(1,10000)
        W2 = np.random.rand(1,10000)
        W3 = np.random.rand(1,10000)
        y=0

      j=j+1

    Cx = C
    CCC = Cx.reshape(n,m)
    plt.imshow(CCC, cmap='hot', interpolation='nearest')
    plt.show()
    return